In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import joblib

# === Rutas ===
ruta = r"C:\Users\uzgre\Codes\Python\Datathon\Reto Oxxo"
archivo = os.path.join(ruta, "train_w_label.csv")

# === Lectura del archivo ===
with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data = pd.read_csv(f)

# === Limpieza inicial ===
if 'Unnamed: 0' in Data.columns:
    Data = Data.drop(columns=['Unnamed: 0'])

Data = Data.drop(columns=['TIENDA_ID', 'DATASET'])

# === MTS2VENTAS_NUM: imputación por grupo (PLAZA_CVE), luego global ===
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].replace(0.0, np.nan)
media_global = Data['MTS2VENTAS_NUM'].mean()
Data['MTS2VENTAS_NUM'] = Data.groupby('PLAZA_CVE')['MTS2VENTAS_NUM'].transform(lambda x: x.fillna(x.mean()))
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].fillna(media_global)

# === NIVEL SOCIOECONÓMICO: codificación ordinal ===
orden_nivel = {'A': 1, 'AB': 2, 'B': 3, 'BC': 4, 'C': 5, 'CD': 6, 'D': 7}
Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].astype(str).map(orden_nivel)
Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].fillna(4)

# === Codificación semántica de variables categóricas ===
cat_vars = ['SEGMENTO_MAESTRO_DESC', 'ENTORNO_DES', 'LID_UBICACION_TIENDA']
Data[cat_vars] = Data[cat_vars].fillna("Missing")
Data['TEXTO_CATEGORICO'] = Data[cat_vars].agg(" ".join, axis=1)

# Vectorización tipo NLP
tfidf = TfidfVectorizer()
X_text = tfidf.fit_transform(Data['TEXTO_CATEGORICO'])

# Reducción de dimensionalidad
svd = TruncatedSVD(n_components=10, random_state=42)
X_text_reduced = svd.fit_transform(X_text)

# Guardamos modelos para futura inferencia en test
joblib.dump(tfidf, os.path.join(ruta, 'tfidf_vectorizer.pkl'))
joblib.dump(svd, os.path.join(ruta, 'svd_reductor.pkl'))

# Integración al DataFrame
for i in range(X_text_reduced.shape[1]):
    Data[f'txt_feat_{i}'] = X_text_reduced[:, i]

# Limpiar columnas originales
Data = Data.drop(columns=cat_vars + ['TEXTO_CATEGORICO'])

# Eliminar duplicados y guardar
Data = Data.drop_duplicates()
Data.to_csv(os.path.join(ruta, "Dataset_Train_limpio2.csv"), index=False)

print("✅ Dataset de entrenamiento procesado con representación semántica. TF-IDF y SVD guardados.")


✅ Dataset de entrenamiento procesado con representación semántica. TF-IDF y SVD guardados.


In [2]:
import numpy as np
import pandas as pd
import os
from joblib import load

# === Rutas ===
ruta = r"C:\Users\uzgre\Codes\Python\Datathon\Reto Oxxo"
archivo = os.path.join(ruta, "test_w_label.csv")

# === Lectura del archivo ===
with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data = pd.read_csv(f)

# === Limpieza inicial ===
if 'Unnamed: 0' in Data.columns:
    Data = Data.drop(columns=['Unnamed: 0'])

Data = Data.drop(columns=['TIENDA_ID', 'DATASET'])

# === MTS2VENTAS_NUM: imputación como en train ===
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].replace(0.0, np.nan)
media_global = Data['MTS2VENTAS_NUM'].mean()
Data['MTS2VENTAS_NUM'] = Data.groupby('PLAZA_CVE')['MTS2VENTAS_NUM'].transform(lambda x: x.fillna(x.mean()))
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].fillna(media_global)

# === NIVEL SOCIOECONÓMICO: codificación ordinal ===
orden_nivel = {'A': 1, 'AB': 2, 'B': 3, 'BC': 4, 'C': 5, 'CD': 6, 'D': 7}
Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].astype(str).map(orden_nivel)
Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].fillna(4)

# === Codificación semántica (TF-IDF + SVD entrenados en train) ===
cat_vars = ['SEGMENTO_MAESTRO_DESC', 'ENTORNO_DES', 'LID_UBICACION_TIENDA']
Data[cat_vars] = Data[cat_vars].fillna("Missing")
Data['TEXTO_CATEGORICO'] = Data[cat_vars].agg(" ".join, axis=1)

# Cargar modelos entrenados
tfidf = load(os.path.join(ruta, 'tfidf_vectorizer.pkl'))
svd = load(os.path.join(ruta, 'svd_reductor.pkl'))

# Aplicar transformación
X_text = tfidf.transform(Data['TEXTO_CATEGORICO'])
X_text_reduced = svd.transform(X_text)

# Integrar nuevas columnas al DataFrame
for i in range(X_text_reduced.shape[1]):
    Data[f'txt_feat_{i}'] = X_text_reduced[:, i]

# Limpiar columnas originales
Data = Data.drop(columns=cat_vars + ['TEXTO_CATEGORICO'])

# Eliminar duplicados y guardar
Data = Data.drop_duplicates()
Data.to_csv(os.path.join(ruta, "Dataset_Test_limpio2.csv"), index=False)

print("✅ Dataset de prueba procesado con representación semántica. Listo para predecir.")



✅ Dataset de prueba procesado con representación semántica. Listo para predecir.
